## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-28-14-50-48 +0000,wapo,What we know about the Minneapolis Catholic sc...,https://www.washingtonpost.com/nation/2025/08/...
1,2025-08-28-14-49-37 +0000,nyt,Mississippi Museum Acquires Gun Linked to Emme...,https://www.nytimes.com/2025/08/28/us/emmett-t...
2,2025-08-28-14-45-00 +0000,wsj,Opinion | E.J. Antoni Is the Right Man for BLS,https://www.wsj.com/opinion/e-j-antoni-is-the-...
3,2025-08-28-14-45-00 +0000,wsj,"Opinion | Speak Now, or Forever Hold Your Peac...",https://www.wsj.com/opinion/speak-now-or-forev...
4,2025-08-28-14-43-04 +0000,nyt,"Gérard Chaliand, Intrepid Authority on Geopoli...",https://www.nytimes.com/2025/08/27/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2357/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
62,trump,55
2,school,19
0,minneapolis,19
3,shooting,16
512,new,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
173,2025-08-28-01-00-00 +0000,wsj,"In his first term, administration officials re...",https://www.wsj.com/politics/policy/in-trumps-...,146
17,2025-08-28-14-18-31 +0000,cbc,U.S. Fed governor Lisa Cook is suing Trump adm...,https://www.cbc.ca/news/world/trump-lisa-cook-...,118
20,2025-08-28-14-08-00 +0000,wsj,Federal Reserve governor Lisa Cook sued Presid...,https://www.wsj.com/economy/central-banking/li...,115
9,2025-08-28-14-35-26 +0000,bbc,US Fed Governor Lisa Cook sues Trump over atte...,https://www.bbc.com/news/articles/c1dxl6ry4y3o...,111
29,2025-08-28-13-46-00 +0000,wsj,"President Trump fired Robert Primus, a board m...",https://www.wsj.com/politics/policy/trump-fire...,104


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
173,146,2025-08-28-01-00-00 +0000,wsj,"In his first term, administration officials re...",https://www.wsj.com/politics/policy/in-trumps-...
54,82,2025-08-28-11-59-49 +0000,nypost,Minneapolis school shooter Robin Westman confe...,https://nypost.com/2025/08/28/us-news/minneapo...
151,77,2025-08-28-03-19-08 +0000,nypost,Russian ballistic missiles hit Kyiv in large-s...,https://nypost.com/2025/08/27/world-news/russi...
9,56,2025-08-28-14-35-26 +0000,bbc,US Fed Governor Lisa Cook sues Trump over atte...,https://www.bbc.com/news/articles/c1dxl6ry4y3o...
146,44,2025-08-28-04-20-00 +0000,wsj,"Susan Monarez, the director of the Centers for...",https://www.wsj.com/politics/policy/cdc-direct...
39,43,2025-08-28-13-03-00 +0000,bbc,Minneapolis mourns two children killed in shoo...,https://www.bbc.com/news/articles/cy40znde9eko...
241,38,2025-08-27-21-01-42 +0000,nypost,Angry Cracker Barrel workers dish on skimpy pa...,https://nypost.com/2025/08/27/business/angry-c...
176,35,2025-08-28-00-45-00 +0000,wsj,Tulsi Gabbard surprised CIA officials last wee...,https://www.wsj.com/politics/national-security...
42,34,2025-08-28-13-01-00 +0000,wsj,Denmark Summons U.S. Envoy Over Suspected Amer...,https://www.wsj.com/world/suspected-american-i...
29,33,2025-08-28-13-46-00 +0000,wsj,"President Trump fired Robert Primus, a board m...",https://www.wsj.com/politics/policy/trump-fire...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
